In [1]:
import numpy as np
import pandas as pd
import graphviz
from IPython.display import IFrame
from PIL import Image
import itertools
from netwerk import Netwerk
from activatie_functies import ActivatieFuncties
from sklearn.datasets import load_iris

## P1.1, 1.3, 1.4: opzet netwerk

Voor het netwerk heb ik gekozen om in plaats van elke neuron een object te maken, gewoon alle weights en biases in matrices en vectoren te zetten, respectievelijk. Een netwerk evalueren aan de hand van een input vector werkt als volgt:

$${g(x):= f^{L}(W^{L}f^{L-1}(W^{L-1}\cdots f^{1}(W^{1}x+b^{1})\cdots )+b^{L-1})+b^{L})}$$
waarbij:
 - $x:$ input vector
 - $L:$ aantal layers
 - $W^{l}=(w_{jk}^{l}):$ matrix van weights tussen layer $l$ en $l-1$, waarbij $w_{jk}^{l}$ de weight tussen node $j$ in layer $l$ en node $k$ in layer $l-1$ 
 - $b^{l}:$ bias vector van layer $l$
 - $f^{l}(x):$ activatiefunctie van layer $l$
 
dit kunnen wij versimpelen door de bias vector aan het respectievelijke weight matrix te plakken en een 1 aan de input; een  bias is immers gewoon een weight die altijd geactiveerd wordt.
het netwerk wordt dus geëvalueert dmv:

$${g(x):= f^{L}((W^{L}|b^{L})f^{L-1}((W^{L-1}|b^{L-1})\cdots f^{1}((W^{1}|b^{1})(x|1))\cdots )))}$$
 

## P1.2: perceptron test

een perceptron is praktisch gewoon een netwerk met maar een layer met een enkele neuron met de `step` als activatiefunctie.
enkele gates als voorbeeld voor de functionaliteit van de perceptron hieronder:

In [2]:
table_in_2 = np.array(list(itertools.product([0, 1], repeat=1)))

table_in_4   = np.array(list(itertools.product([0, 1], repeat=2)))

table_in_8   = np.array(list(itertools.product([0, 1], repeat=3)))

In [3]:
netw = Netwerk(0, 0, 0, 0, ActivatieFuncties.STEP)

netw._weights = [np.array([[ 1, 1,-2]])]
print("truth table AND")
for x in table_in_4:
    print(str(x) + " -> " + str(netw.evaluate(x)))
    
netw._weights = [np.array([[ 1, 1,-1]])]
print("\ntruth table OR")
for x in table_in_4:
    print(str(x) + " -> " + str(netw.evaluate(x)))
    
netw._weights = [np.array([[-1, 0]])]
print("\ntruth table NOT")
for x in table_in_2:
    print(str(x) + " -> " + str(netw.evaluate(x)))

netw._weights = [np.array([[-1,-1,-1, 0]])]
print("\ntruth table NOR w/ 3 in")
for x in table_in_8:
    print(str(x) + " -> " + str(netw.evaluate(x)))
    
netw._weights = [np.array([[ 1, 1, 1,-1]])]
print("\ntruth table NAND w/ 3 in")
for x in table_in_8:
    print(str(x) + " -> " + str(netw.evaluate(x)))
    
g = netw.visualise_network(np.array(['x1', 'x2', 'x3']), mindiam=.5, minlen=3, titel='NAND-gate perceptron', filename='NAND')
g.render(directory='graphviz_renders', view=False)
im = Image.open('./graphviz_renders/NAND.gv.bmp')
IFrame('./graphviz_renders/NAND.gv.bmp', width=im.size[0], height=im.size[1])

truth table AND
[0 0] -> [[0]]
[0 1] -> [[0]]
[1 0] -> [[0]]
[1 1] -> [[1]]

truth table OR
[0 0] -> [[0]]
[0 1] -> [[1]]
[1 0] -> [[1]]
[1 1] -> [[1]]

truth table NOT
[0] -> [[1]]
[1] -> [[0]]

truth table NOR w/ 3 in
[0 0 0] -> [[1]]
[0 0 1] -> [[0]]
[0 1 0] -> [[0]]
[0 1 1] -> [[0]]
[1 0 0] -> [[0]]
[1 0 1] -> [[0]]
[1 1 0] -> [[0]]
[1 1 1] -> [[0]]

truth table NAND w/ 3 in
[0 0 0] -> [[0]]
[0 0 1] -> [[1]]
[0 1 0] -> [[1]]
[0 1 1] -> [[1]]
[1 0 0] -> [[1]]
[1 0 1] -> [[1]]
[1 1 0] -> [[1]]
[1 1 1] -> [[1]]


## P1.5: netwerk test

Om het perceptron netwerk te demonstreren hieronder een netwerk met de functionaliteit van een half-adder:

In [4]:
#`weights` en `biases` is private dus dit mag eigenlijk niet, maar je zou in het echt toch nooit 
                            #zelf de weights en biases zetten

netw = Netwerk(0, 0, 0, 0, ActivatieFuncties.STEP)
netw._weights = [np.array([[ 1, 1,-2],
                           [ 1, 1,-1],
                           [-1,-1, 1]]),
                 np.array([[ 1, 0, 0,-1],
                           [ 0, 1, 1,-2]])]

print("truth table XOR") #een XOR is eigenlijk gewoon het sum gedeelte van een half-adder, dus alleen dat gedeelte van
                         #de output van de half-adder nemen resulteert in de truth table van een XOR
for x in table_in_4:
    print(str(x) + " -> " + str(netw.evaluate(x)[0][1]))

print("\ntruth table half-adder: ")
for x in table_in_4:
    print(str(x) + " -> " + str(netw.evaluate(x)))
    
g = netw.visualise_network(np.array(['x1', 'x2']), mindiam=1.2, minlen=4, titel='Half-adder perceptron netwerk', filename='HalfAdder')
g.render(directory='graphviz_renders', view=False)
im = Image.open('./graphviz_renders/HalfAdder.gv.bmp')
IFrame('./graphviz_renders/HalfAdder.gv.bmp', width=im.size[0], height=im.size[1])

truth table XOR
[0 0] -> 0
[0 1] -> 1
[1 0] -> 1
[1 1] -> 0

truth table half-adder: 
[0 0] -> [[0 0]]
[0 1] -> [[0 1]]
[1 0] -> [[0 1]]
[1 1] -> [[1 0]]


## P2: Perceptron learning rule

Voor deze opdracht heb ik de functie `update_trivial` gemaakt, die één enkele layer updated aan de hand van:
$$Δw_j = η (target^{(i)} – output^{(i)}) x_j^{(i)}$$
Wat in onze enkele layer dus neerkomt op:
$$W_1 = W_0 + η (target^{(i)} – f(W_0in^{(i)}))⊗_{outer}in^{(i)}$$
waarbij:
 - $W_n:$ matrix van weights (incl. biases) na update n 
 - $η:$ learning rate
 - $target^{(i)}:$ target feature i 
 - $in^{(i)}:$ input feature i (incl. 1 voor bias)
 - $f(x):$ activatiefunctie

## P2.3 a: AND 
hieronder de learning rule gedemonstreerd door een perceptron de AND-gate te leren:

In [5]:
np.random.seed(1819772)
netw = Netwerk(0, 0, 2, 1, ActivatieFuncties.STEP, .8)

d_and = np.array([[0], [0], [0], [1]])

print("\ninitial weights:")
for w in netw._weights:
    print(w)
print("initial MSE:")
print(netw.loss_MSE(table_in_4, d_and))
print()

for i in range(4):
    netw.update_trivial(table_in_4, d_and, True)

g1 = netw.visualise_network(np.array(['x1', 'x2']), mindiam=.5, minlen=5, filename='learningRule')
g1.render(directory='graphviz_renders', view=False)
im = Image.open('./graphviz_renders/learningRule.gv.bmp')
IFrame('./graphviz_renders/learningRule.gv.bmp', width=im.size[0], height=im.size[1])


initial weights:
[[-1.34322705  0.53339359  1.04945582]]
initial MSE:
[0.5]

Δ ⊗ in    = 
[[-0.  -0.  -0.8]]
updated W = 
[[[-1.34322705  0.53339359  0.24945582]]]
MSE       = [0.75]

Δ ⊗ in    = 
[[-0.  -0.8 -0.8]]
updated W = 
[[[-1.34322705 -0.26660641 -0.55054418]]]
MSE       = [0.25]

Δ ⊗ in    = 
[[0. 0. 0.]]
updated W = 
[[[-1.34322705 -0.26660641 -0.55054418]]]
MSE       = [0.25]

Δ ⊗ in    = 
[[0.8 0.8 0.8]]
updated W = 
[[[-0.54322705  0.53339359  0.24945582]]]
MSE       = [0.5]

Δ ⊗ in    = 
[[-0.  -0.  -0.8]]
updated W = 
[[[-0.54322705  0.53339359 -0.55054418]]]
MSE       = [0.25]

Δ ⊗ in    = 
[[0. 0. 0.]]
updated W = 
[[[-0.54322705  0.53339359 -0.55054418]]]
MSE       = [0.25]

Δ ⊗ in    = 
[[0. 0. 0.]]
updated W = 
[[[-0.54322705  0.53339359 -0.55054418]]]
MSE       = [0.25]

Δ ⊗ in    = 
[[0.8 0.8 0.8]]
updated W = 
[[[0.25677295 1.33339359 0.24945582]]]
MSE       = [0.75]

Δ ⊗ in    = 
[[-0.  -0.  -0.8]]
updated W = 
[[[ 0.25677295  1.33339359 -0.55054418]]]
MSE    

## P2.3 b: XOR
een enkele perceptron kan nooit een XOR-gate leren

In [6]:
np.random.seed(1819772)
netw = Netwerk(0, 0, 2, 1, ActivatieFuncties.STEP, .8)

d_xor = np.array([[0], [1], [1], [0]])

print("\ninitial weights:")
for w in netw._weights:
    print(w)
print("initial MSE:")
print(netw.loss_MSE(table_in_4, d_xor))
print()

for i in range(10):
    netw.update_trivial(table_in_4, d_xor, True)

g1 = netw.visualise_network(np.array(['x1', 'x2']), mindiam=.5, minlen=5, filename='learningRule')
g1.render(directory='graphviz_renders', view=False)
im = Image.open('./graphviz_renders/learningRule.gv.bmp')
IFrame('./graphviz_renders/learningRule.gv.bmp', width=im.size[0], height=im.size[1])


initial weights:
[[-1.34322705  0.53339359  1.04945582]]
initial MSE:
[0.75]

Δ ⊗ in    = 
[[-0.  -0.  -0.8]]
updated W = 
[[[-1.34322705  0.53339359  0.24945582]]]
MSE       = [0.5]

Δ ⊗ in    = 
[[0. 0. 0.]]
updated W = 
[[[-1.34322705  0.53339359  0.24945582]]]
MSE       = [0.5]

Δ ⊗ in    = 
[[0.8 0.  0.8]]
updated W = 
[[[-0.54322705  0.53339359  1.04945582]]]
MSE       = [0.5]

Δ ⊗ in    = 
[[-0.8 -0.8 -0.8]]
updated W = 
[[[-1.34322705 -0.26660641  0.24945582]]]
MSE       = [0.75]

Δ ⊗ in    = 
[[-0.  -0.  -0.8]]
updated W = 
[[[-1.34322705 -0.26660641 -0.55054418]]]
MSE       = [0.5]

Δ ⊗ in    = 
[[0.  0.8 0.8]]
updated W = 
[[[-1.34322705  0.53339359  0.24945582]]]
MSE       = [0.5]

Δ ⊗ in    = 
[[0.8 0.  0.8]]
updated W = 
[[[-0.54322705  0.53339359  1.04945582]]]
MSE       = [0.5]

Δ ⊗ in    = 
[[-0.8 -0.8 -0.8]]
updated W = 
[[[-1.34322705 -0.26660641  0.24945582]]]
MSE       = [0.75]

Δ ⊗ in    = 
[[-0.  -0.  -0.8]]
updated W = 
[[[-1.34322705 -0.26660641 -0.55054418]]]

## P2.3 c: Iris
met alleen *Setosa* en *Versicolour* lijkt het wel te werken, maar met *Versicolour* en *Verginica* niet

In [7]:
iris = load_iris()
X = iris.data[50:]
y = iris.target[50:]
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [8]:
np.random.seed(1819772)
netw = Netwerk(0, 0, 4, 1, ActivatieFuncties.STEP, .8)

iris = load_iris()
X = iris.data[:100]
y = np.reshape(iris.target[:100],(-1,1))

for i in range(5):
    netw.update_trivial(X, y, False)
    print(netw._weights)
    print(netw.loss_MSE(X, y))

[[[ 4.25677295  3.09339359  4.80945582  2.14135062 -0.55814302]]]
[0.5]
[[[ 1.85677295  0.45339359  6.32945582  2.94135062 -1.35814302]]]
[0.5]
[[[ 1.85677295 -0.34660641  7.60945582  3.66135062 -1.35814302]]]
[0.5]
[[[-2.22322705 -3.14660641  6.48945582  3.50135062 -2.15814302]]]
[0.]
[[[-2.22322705 -3.14660641  6.48945582  3.50135062 -2.15814302]]]
[0.]


In [9]:
np.random.seed(1819772)
netw = Netwerk(0, 0, 4, 1, ActivatieFuncties.STEP, .8)

X = iris.data[50:]
y = np.reshape(iris.target[50:],(-1,1))

for i in range(10):
    netw.update_trivial(X, y, False)
    print(netw._weights)
    print(netw.loss_MSE(X, y))

[[[267.77677295 122.05339359 226.88945582  83.18135062  39.44185698]]]
[0.5]
[[[531.29677295 241.01339359 448.96945582 164.22135062  79.44185698]]]
[0.5]
[[[794.81677295 359.97339359 671.04945582 245.26135062 119.44185698]]]
[0.5]
[[[1058.33677295  478.93339359  893.12945582  326.30135062  159.44185698]]]
[0.5]
[[[1321.85677295  597.89339359 1115.20945582  407.34135062  199.44185698]]]
[0.5]
[[[1585.37677295  716.85339359 1337.28945582  488.38135062  239.44185698]]]
[0.5]
[[[1848.89677295  835.81339359 1559.36945582  569.42135062  279.44185698]]]
[0.5]
[[[2112.41677295  954.77339359 1781.44945582  650.46135062  319.44185698]]]
[0.5]
[[[2375.93677295 1073.73339359 2003.52945582  731.50135062  359.44185698]]]
[0.5]
[[[2639.45677295 1192.69339359 2225.60945582  812.54135062  399.44185698]]]
[0.5]


## P3.2: Neuron Unit

zoals hieronder te zien werkt de neuron niet met dezelfde weights als de perceptron, als de uitkomst wordt afgerond zit het er echter wel in de buurt. de Sigmoid komt immers niet exact 0 of 1 uit, maar iets daar tussenin.

In [10]:
netw = Netwerk(0, 0, 0, 0, ActivatieFuncties.SIGMOID)

netw._weights = [np.array([[ 1, 1,-2]])]
print("truth table AND")
for x in table_in_4:
    print(str(x) + " -> " + str(netw.evaluate(x)))
    
netw._weights = [np.array([[ 1, 1,-1]])]
print("\ntruth table OR")
for x in table_in_4:
    print(str(x) + " -> " + str(netw.evaluate(x)))
    
netw._weights = [np.array([[-1, 0]])]
print("\ntruth table NOT")
for x in table_in_2:
    print(str(x) + " -> " + str(netw.evaluate(x)))

truth table AND
[0 0] -> [[0.11920292]]
[0 1] -> [[0.26894142]]
[1 0] -> [[0.26894142]]
[1 1] -> [[0.5]]

truth table OR
[0 0] -> [[0.26894142]]
[0 1] -> [[0.5]]
[1 0] -> [[0.5]]
[1 1] -> [[0.73105858]]

truth table NOT
[0] -> [[0.5]]
[1] -> [[0.26894142]]


als we de weights zodanig veranderen dat de uitkomst helemaal links of rechts van de sigmoid zit wordt de uitkomst al correcter. ik heb hier voor meervouden van 6 gekozen, gezien σ(6) = 0.99, wat close enough by de 1 zit voor deze doeleinden.

In [11]:
netw = Netwerk(0, 0, 0, 0, ActivatieFuncties.SIGMOID)

netw._weights = [np.array([[ 12, 12, -18]])]
print("truth table AND")
for x in table_in_4:
    print(str(x) + " -> " + str(netw.evaluate(x)))
    
netw._weights = [np.array([[ 12, 12, -6]])]
print("\ntruth table OR")
for x in table_in_4:
    print(str(x) + " -> " + str(netw.evaluate(x)))
    
netw._weights = [np.array([[-12, 6]])]
print("\ntruth table NOT")
for x in table_in_2:
    print(str(x) + " -> " + str(netw.evaluate(x)))
    


truth table AND
[0 0] -> [[1.52299795e-08]]
[0 1] -> [[0.00247262]]
[1 0] -> [[0.00247262]]
[1 1] -> [[0.99752738]]

truth table OR
[0 0] -> [[0.00247262]]
[0 1] -> [[0.99752738]]
[1 0] -> [[0.99752738]]
[1 1] -> [[0.99999998]]

truth table NOT
[0] -> [[0.99752738]]
[1] -> [[0.00247262]]


In [12]:
netw._weights = [np.array([[-12,-12,-12, 6]])]
print("\ntruth table NOR w/ 3 in")
for x in table_in_8:
    print(str(x) + " -> " + str(netw.evaluate(x)))
    
g = netw.visualise_network(np.array(['x1', 'x2', 'x3']), mindiam=.5, minlen=3, titel='NOR-gate neuron', filename='NOR')
g.render(directory='graphviz_renders', view=False)
im = Image.open('./graphviz_renders/NOR.gv.bmp')
IFrame('./graphviz_renders/NOR.gv.bmp', width=im.size[0], height=im.size[1])


truth table NOR w/ 3 in
[0 0 0] -> [[0.99752738]]
[0 0 1] -> [[0.00247262]]
[0 1 0] -> [[0.00247262]]
[0 1 1] -> [[1.52299795e-08]]
[1 0 0] -> [[0.00247262]]
[1 0 1] -> [[1.52299795e-08]]
[1 1 0] -> [[1.52299795e-08]]
[1 1 1] -> [[9.35762297e-14]]


In [13]:
netw = Netwerk(0, 0, 0, 0, ActivatieFuncties.SIGMOID)
netw._weights = [np.array([[ 12, 12,-18],
                           [ 12, 12,-6],
                           [-12,-12, 18]]),
                 np.array([[ 12, 0, 0,-6],
                           [ 0, 12, 12,-18]])]

print("\ntruth table half-adder: ")
for x in table_in_4:
    print(str(x) + " -> " + str(netw.evaluate(x)))
    
g = netw.visualise_network(np.array(['x1', 'x2']), mindiam=1.2, minlen=4, titel='Half-adder neuraal netwerk', filename='HalfAdder')
g.render(directory='graphviz_renders', view=False)
im = Image.open('./graphviz_renders/HalfAdder.gv.bmp')
IFrame('./graphviz_renders/HalfAdder.gv.bmp', width=im.size[0], height=im.size[1])


truth table half-adder: 
[0 0] -> [[0.00247262 0.0025469 ]]
[0 1] -> [[0.0025469 0.9973766]]
[1 0] -> [[0.0025469 0.9973766]]
[1 1] -> [[0.9974531 0.0025469]]
